In [1]:
import pandas as pd
import numpy as np
import os.path
from sklearn.cluster import KMeans
from sklearn.preprocessing import *
import warnings
warnings.filterwarnings('ignore')

### Read Data

In [2]:
inv_data = pd.read_csv('dat_cluster.csv')

### Clustering Function

In [3]:
def clustering(x,n):
    kmeans = KMeans(n_clusters=n, init='k-means++', max_iter=300, n_init=10, random_state=0)
    clusters=[]
    x_2 = x[['total_inventory']]
    clusters = kmeans.fit(x_2)
    x['Cluster'] = clusters.labels_
    return x

### Run cluser at style level

In [4]:
#List of styles
lst_styles  = list(inv_data['sku_id_wo_store'].unique())

In [25]:
len(lst_styles)

201

In [21]:
cluster_df

,sku_id_wo_store,total_inventory,Cluster
0,58574384,9.0,0
1,58574384,8.0,1
2,58574384,14.0,2
3,58574384,2.0,3
4,58576606,1.0,0
...,...,...,...
12811,85425949,9.0,5
12812,85425949,8.0,5
12813,85425949,61.0,8
12814,85425949,7.0,6


In [7]:
cluster_df['Cluster'].nunique()

10

In [9]:
cluster_df.loc[cluster_df['Cluster'] == 0]

,sku_id_wo_store,store,total_inventory,Cluster
0,58574384,USRO0121,9.0,0
8,58576606,USRO0124,2.0,0
9,58578503,USRO0005,1.0,0
11,58578503,USRO0016,1.0,0
12,58578503,USRO0023,1.0,0
...,...,...,...,...
12746,85425949,USRO0021,22.0,0
12753,85425949,USRO0031,26.0,0
12798,85425949,USRO0102,24.0,0
12802,85425949,USRO0106,26.0,0


In [27]:
cluster_df.loc[cluster_df['Cluster'] == 1]

,sku_id_wo_store,total_inventory,Cluster
1,58574384,8.0,1
5,58576606,11.0,1
15,58578503,6.0,1
48,58610901,121.0,1
53,58610901,119.0,1
...,...,...,...
12749,85425949,42.0,1
12783,85425949,40.0,1
12793,85425949,41.0,1
12806,85425949,38.0,1


In [11]:
!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 5.5 MB/s 


In [12]:
import faiss
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [26]:
def faiss_clustering(df,K_Faiss = 8):
    x_df = df
    x_df = pd.DataFrame(inv_data[inv_data['sku_id_wo_store']==style][['sku_id_wo_store','store','total_inventory']].drop_duplicates())
    x = StandardScaler().fit_transform(x_df.drop(["sku_id_wo_store","store"], axis=1).astype(np.float64))
    pca = PCA(.95)
    principalComponents = pca.fit_transform(X = x)
    x = principalComponents
    x_f = np.ascontiguousarray(x)
    n_init = 10 #equivalent to scikit learn
    max_iter = 300 #equivalent to scikit learn
    kmeans = faiss.Kmeans(d=x_f.shape[1], k = int(K_Faiss), niter=max_iter, nredo=n_init)
    kmeans.train(x_f.astype(np.float32))
    x_df['Cluster'] = kmeans.index.search(x_f.astype(np.float32), 1)[1]
    cluster_df = x_df[["sku_id_wo_store","total_inventory","Cluster"]]
    return cluster_df
    

In [20]:
cluster_df = pd.DataFrame()
#If available points are less than 10, no cluster and if points more than 10, cluster to 10
for style in lst_styles:
    temp_df = pd.DataFrame(inv_data[inv_data['sku_id_wo_store']==style][['sku_id_wo_store','store','total_inventory']].drop_duplicates())
    if temp_df.shape[0]>=10:
        temp_df = faiss_clustering(temp_df,10)
    else:
        temp_df = faiss_clustering(temp_df,temp_df.shape[0])
    cluster_df = cluster_df.append(temp_df) 